In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time

finaldf= pd.DataFrame()

In [20]:
final2df= pd.DataFrame()

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

def start_driver():
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    return webdriver.Chrome(options=options)

driver = start_driver()
data = []

TOTAL_PAGES = 21
RESTART_EVERY = 10

for page in range(1, 2):

    if page % RESTART_EVERY == 0:
        driver.quit()
        driver = start_driver()
        time.sleep(2)

    url = f"https://www.zameen.com/Homes/Karachi_Gadap_Town_Surjani_Town_Sector_1-6727-1.html"
    print(f"Scraping page {page}...")

    try:
        driver.get(url)

        WebDriverWait(driver, 12).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "li[role='article']"))
        )

        soup = BeautifulSoup(driver.page_source, "lxml")
        listings = soup.select("li[role='article']")
        print(f"  Listings found: {len(listings)}")

        for li in listings:
            price = li.select_one("span.dc381b54")
            price = price.text.strip() if price else "N/A"

            beds = baths = area = "N/A"
            info = li.select("div.f0835674 span[aria-label]")

            for span in info:
                label = span.get("aria-label")
                if label == "Beds":
                    beds = span.text.strip()
                elif label == "Baths":
                    baths = span.text.strip()
                elif label == "Area":
                    area = span.text.strip()

            data.append({
                "Price": price,
                "Beds": beds,
                "Baths": baths,
                "Area": area,
                "Location score": 1
            })

    except Exception as e:
        print(f"  ❌ Error on page {page}: {e}")
        continue

driver.quit()

df = pd.DataFrame(data)
final2df= pd.concat([final2df,df],ignore_index=True)
final2df.to_csv("homes_data.csv", index=False, encoding="utf-8-sig")

print("Scraping completed successfully ✅")
print(final2df.shape)


Scraping page 1...
  Listings found: 25
Scraping completed successfully ✅
(225, 5)
